### [Course Schedule](https://leetcode.com/problems/course-schedule/)

There are a total of n courses you have to take, labeled from 0 to n-1.

Some courses may have prerequisites, for example to take course 0 you have to first take course 1, which is expressed as a pair: [0,1]

Given the total number of courses and a list of prerequisite pairs, is it possible for you to finish all courses?

**Example 1:**
```
Input: 2, [[1,0]] 
Output: true
```

Explanation: There are a total of 2 courses to take. 
             To take course 1 you should have finished course 0. So it is possible.
             
**Example 2:**
```
Input: 2, [[1,0],[0,1]]
Output: false
```

Explanation: There are a total of 2 courses to take. 
             To take course 1 you should have finished course 0, and to take course 0 you should
             also have finished course 1. So it is impossible.
             
**Note:**

- The input prerequisites is a graph represented by a list of edges, not adjacency matrices. Read more about how a graph is represented.
- You may assume that there are no duplicate edges in the input prerequisites.

In [ ]:
from collections import deque

class Solution(object):
    def canFinish(self, numCourses, prerequisites, useBFS=True):
        """
        :type numCourses: int
        :type prerequisites: List[List[int]]
        :type useBFS: bool
        :rtype: bool
        """
        if useBFS:
            return self.canFinishBFS(numCourses, prerequisites)
        else:
            return self.canFinishDFS(numCourses, prerequisites)
        
    def canFinishDFS(self, numCourses, prerequisites):
        """
        :type numCourses: int
        :type prerequisites: List[List[int]]
        :rtype: bool
        """
        # previously we solved using breadth first search by visiting the neigbor nodes
        # lets try depth first search this time.
        
        # with DFS, we keep track of the visited nodes along a path. if there is a
        # cycle, we will come back to one of the nodes already visited. we clear
        # the visited state after crossing a path from each vertex.
        
        def isCyclic(vertex, graph, visited, inPath):
            
            if not visited[vertex]:
                
                visited[vertex] = True
                inPath[vertex] = True

                for neighbor in graph[vertex]:
                    # Two possible cases for cycle
                    # 1. neighbor is not visited yet.. and turns to have cyclic
                    # 2. neighbor is already visited, there is no cycle from that path, but one
                    #    exists in the current path
                    if (not visited[neighbor] and isCyclic(neighbor, graph, visited, inPath)):
                        return True
                    elif inPath[neighbor]:
                        return True
                    # if no cycle exists from the current neigbor, move on to the next
                
            inPath[vertex] = False
            return False
        
        # edge cases
        # 0 or 1 course
        if numCourses < 2:
            # No depedencies
            return True
        
        graph = self._buildGraph(numCourses, prerequisites)
        
        # tracker lists
        visited = [False] * numCourses
        inPath = [False] * numCourses
        
        for course in range(numCourses):
            if isCyclic(course, graph, visited, inPath):
                return False
        
        return True
                
        
    def _buildGraph(self, numVertices, edges):
            graph = [[] for _ in range(numVertices)]
            
            for edge in edges:
                src, dest = edge
                graph[src].append(dest)
                
            return graph
        
    def _buildInDegree(self, graph):
            numVertices = len(graph)
            indegree = [0] * numVertices
            for vertex in range(numVertices):
                for neighbor in graph[vertex]:
                    indegree[neighbor] += 1
            
            return indegree
                    
    def canFinishBFS(self, numCourses, prerequisites):
        """
        :type numCourses: int
        :type prerequisites: List[List[int]]
        :rtype: bool
        """
        
        # number of courses..0..n-1
        # prerequisites: [c1, c2] c1 dependent on c2, 
        #   c1, c2 in range of 0..n-1
        
        # given a list of prerequisites, can you take all courses
        # ranging from 0..n-1?
        
        # number of courses = number of vertices
        # number of prereq pairs = number of edges 
        
        # two courses are in deadlock if they are dependent on each other
        # a cyclic relation. can treat this as a directed graph
        # If the graph is directed acyclic graph, then all prereqs can be met
        
        # prerequisites - list of edges in a graph
        
        # finding cycle using Kahn's algorithm of topology sort method
        
        # algorithm
        #   1. find the in-degree for each vertex
        #   2. add vertex with in-degree as 0 to the queue.. These are the possible starting nodes in the graph
        #   3. for each starting node:
        #       mark this node as visited
        #       for each neighbor of starting node:
        #           decrement in-degree
        #           if in-degree of neighbor reaches 0:
        #               this is also a starting point, if we disconnect the graph at this point.
        #               add this neighbor to queue
        #   4. if all nodes are visited, then there is no cycle.
        #   
        
        # edge cases
        # 0 or 1 course
        if numCourses < 2:
            # No depedencies
            return True
            
        graph = self._buildGraph(numCourses, prerequisites)
        indegree = self._buildInDegree(graph)
        
        starters = deque([course for course in range(numCourses) if indegree[course] == 0])

        attendedCourses = 0
        while starters:
            course = starters.popleft()
            
            # count towards attended courses
            attendedCourses += 1
            
            for prereqCourse in graph[course]:
                # decrement the indegree (i.e. depedency)
                indegree[prereqCourse] -= 1
                if indegree[prereqCourse] == 0:
                    # prereqCourse is also a possible starter course.
                    # add it to our queue
                    starters.append(prereqCourse)
        
        
        return True if attendedCourses == numCourses else False

- Time and Space complexity of both the solutions are O(V+E)
- These solutions are modeled after basic depth-first and bread-first search of directed graphs
- Reference links: [G4G DFS](https://www.geeksforgeeks.org/detect-cycle-in-a-graph/), [G4G BFS](https://www.geeksforgeeks.org/detect-cycle-in-a-directed-graph-using-bfs/), [Kahn's Topology Sort](https://www.geeksforgeeks.org/topological-sorting-indegree-based-solution/)

In [4]:
# now lets run through some tests

tests = {
    "test": [
        {
            "input": {
                "numCourses" : 2,
                "prerequisites": [
                    [1, 0]
                ]
            },
            "output": True
        },
        {
            "input": {
                "numCourses" : 2,
                "prerequisites": [
                    [1, 0],
                    [0, 1]
                ]
            },
            "output": False
        },
        {
            "input": {
                "numCourses" : 3,
                "prerequisites": [
                    [1, 0],
                    [0, 2],
                    [2, 1]
                ]
            },
            "output": False
        },
        {
            "input": {
                "numCourses" : 4,
                "prerequisites": [
                    [0, 1],
                    [1, 2],
                    [2, 3]
                ]
            },
            "output": True
        }
    ]
}

s = Solution()
for test in tests["test"]:
    numCourses = test["input"]["numCourses"]
    prereqs = test["input"]["prerequisites"]
    
    assert(s.canFinish(numCourses, prereqs, useBFS=True) == test["output"])
    assert(s.canFinish(numCourses, prereqs, useBFS=False) == test["output"])